# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# identify file path
weather_csv = '../main_script/weather_data.csv'

# turn csv into dataframe
weather_df = pd.read_csv(weather_csv)

weather_df.head()


,Name,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
0,Fort Smith,35.3859,-94.3986,61.070,1.05,1,29
1,Taquari,-29.7997,-51.8644,69.278,1.70,24,85
2,Grand Gaube,-20.0064,57.6608,81.266,3.13,100,79
3,Albany,42.6001,-73.9662,24.278,3.86,43,52
4,Port Alfred,-33.5906,26.8910,68.270,4.02,100,90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# convert humidity to float, drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df['Humidity'].astype(float)

# store lattitude and longitude together
locations = weather_df[['Latitude', 'Longitude']].astype(float)

# create humidity heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, 
                                     weights = humidity, 
                                     dissipating = False, 
                                     max_intensity = max(weather_df['Humidity']), 
                                     point_radius = 2)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# copy weather_df
ideal_weather_df = weather_df.copy()

# filter through df to find cities with ideal conditions
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Temperature(F)'] < 70) & (ideal_weather_df['Temperature(F)'] > 60)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed(mph)'] < 10)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Cloud Coverage'] < 60) & (ideal_weather_df['Cloud Coverage'] > 10)]

ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df


,Name,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
1,Taquari,-29.7997,-51.8644,69.278,1.70,24,85
19,Hermanus,-34.4187,19.2345,64.274,0.89,11,79
21,Los Llanos de Aridane,28.6585,-17.9182,64.670,9.77,40,68
75,Henties Bay,-22.1160,14.2845,69.638,1.79,13,65
177,Kānker,20.2719,81.4931,67.082,1.18,19,21
225,Necochea,-38.5473,-58.7368,66.272,1.79,11,77
264,Christchurch,-43.5333,172.6333,66.470,6.17,20,45
298,Mīnāb,27.1467,57.0801,64.670,3.09,20,82
326,Kahna Nau,31.3692,74.3653,66.470,2.06,40,63
342,Norton,-17.8833,30.7000,61.268,0.89,39,89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = ideal_weather_df.copy()

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
row_index = 0

# limit hotel search within 5000m of each location
params = {'radius': 5000, 
          'types': 'lodging',
          'key': g_key}

# walk through hotel_df
for i, row in hotel_df.iterrows():
    
    # grab lat and lng data
    lat = row['Latitude']
    lng = row['Longitude']
    
    # add lat and lng data to params dict
    params['location'] = f'{lat}, {lng}'
    
    # API request
    print(f'Request number {row_index} for {row["Name"]}.')
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel: {results[0]['name']}")
        hotel_df.loc[row_index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print(" ")
    
    row_index = row_index + 1

Request number 0 for Taquari.
Closest hotel: Martins Hotel
 
Request number 1 for Hermanus.
Closest hotel: Misty Waves Boutique Hotel
 
Request number 2 for Los Llanos de Aridane.
Closest hotel: Valle Aridane
 
Request number 3 for Henties Bay.
Closest hotel: Fishermans Guest House
 
Request number 4 for Kānker.
Closest hotel: STARiHOTEL Kanker
 
Request number 5 for Necochea.
Closest hotel: dyd -mar Hotel
 
Request number 6 for Christchurch.
Closest hotel: The George
 
Request number 7 for Mīnāb.
Closest hotel: Bam campground
 
Request number 8 for Kahna Nau.
Closest hotel: Ahle hadees masjid kahna nau
 
Request number 9 for Norton.
Closest hotel: Chingwara Mansion
 
Request number 10 for Luk Pu.
Closest hotel: Classic View Hotel
 


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Name</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
